# Meta-heuristieken

In [14]:
import math
import numpy as np
import pandas as pd
from simanneal import Annealer

## Vraag 1: TSP nearest neighbour benadering
Slide 14: Geef de oplossing indien je start bij punt b.

In [15]:
#SOLUTION_START
# (b,c,d,e,a)
#SOLUTION_END

## Vraag 2: De rugzak (aka the knapsack problem)
Je bevindt je in een geheime kamer die uitgerust is met een deur met tijdslot. Je ziet een timer aftellen die meldt dat je nog maar vijf minuten over het alvorens de deur voor altijd op slot zal zijn. Voor je neus liggen waardevolle voorwerpen met elk hun eigen opbrengst en gewicht. Je hebt een rugzak bij die een absoluut maximaal gewicht kan torsen van 750gr.   Op Canvas vind je de lijst van voorwerpen met hun gewicht en opbrengst.  Stel de optimale rugzak samen.  Je zou op een optimale opbrengst van 1458 moeten uitkomen (of toch zeker een waarde dicht daarbij in de buurt).

In [16]:
knapsack_items = pd.read_csv('Knapsack Items.csv', delimiter=',')
knapsack_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     15 non-null     object
 1   gewichten(gr)  15 non-null     int64 
 2   waarde         15 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 492.0+ bytes


In [17]:
#SOLUTION_START

class KnapsackProblem(Annealer):
    def energy(self):
        solution = self.state
        total_weight = (solution * weights_items).sum()
        if total_weight > 750:
            total_value = 0
        else:
            total_value = (solution * values_items).sum()
        return -total_value # - want max

    def move(self):
        i = np.random.randint(0,len(self.state))
        self.state[i] = not self.state[i]

weights_items=knapsack_items['gewichten(gr)']
values_items=knapsack_items['waarde']
init_sol = np.random.randint(0,2,size=len(knapsack_items)) #initial solution
knapsack=KnapsackProblem(init_sol)
knapsack.anneal()

#SOLUTION_END

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


(array([1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]), -1458)

 ## Vraag 3: De dakgoten
Je bent belast met het ontwerp van dakgoten waarbij de productiekost zo laag mogelijk moet zijn. Daarom is het noodzakelijk dat de dakgoten een zo optimale doorsnede hebben met het beschikbare materiaal zodat bladeren en vuil makkelijk afgevoerd kunnen worden.  Het bedrijf waarvoor je werkt koopt metalen platen aan die een breedte hebben van 1m. M.a.w. h + b + h  -zie tekening- moet kleiner of gelijk zijn aan 1m.  Bepaal de ideale breedte B en hoogte H van de dakgoot die je uit de platen van 1m kan maken.

```
  |       |
h |       |
  |_______|
      b
```


In [22]:
#SOLUTION_START

class GutterProblem(Annealer):
    def energy(self):
        b = self.state[0]
        h = (1 - b)/2
        return -b*h  # - want max

    def move(self):
        self.state[0] += np.random.normal(0,0.1)
        self.state[0] = np.clip(self.state[0], 0, 1)
        return

init_sol = np.random.uniform(0,1, size=1) #initial solution
gutter=GutterProblem(init_sol)
gutter.anneal()

#SOLUTION_END

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


(array([0.49998537]), -0.12499999989302947)

## Vraag 4: Voetbalstadium
De plaatselijke sportclub wil een nieuw stadion bouwen.  Het sportveld bestaat uit een rechthoek met 2 halve cirkels rechts en links, zie figuur. De omtrek moet 400m bedragen. Tegelijkertijd willen we ervoor zorgen dat het centrale middenveld (de rechthoek) een maximale oppervlakte heeft.   Bepaal de ideale lengte –en breedteverhouding.

```
                  rechthoek
                 ___________
              / |           | \
halve cirkel |  |B          |  | halve cirkel
              \ |___________| /
                      L
```

In [28]:
#SOLUTION_START

class StadiumProblem(Annealer):
    def energy(self):
        L = self.state[0]
        # circumference = 400 = 2 * L + B * math.pi => B = (400 - 2* L)/math.pi
        B = (400 - 2* L)/math.pi
        return -B*L   # - want max

    def move(self):
        self.state[0] += np.random.normal(0,0.1)
        self.state[0] = np.clip(self.state[0], 0, 200)
        return

init_sol = np.random.uniform(0,200, size=1) #initial solution
stadium=StadiumProblem(init_sol)
stadium.anneal()

#SOLUTION_END

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


(array([100.00003922]), -6366.197723674834)

## Vraag 5: Optimalisatie
Gegeven volgende te maximaliseren doelfunctie:

obj = 0.2 + x^2 + y^2 - 0.1 cos(6*pi*x) - 0.1 cos(6*pi*y)

Met volgende beperkingen:  -1.0 ≤ x ≤  1.0	en -1.0 ≤ y ≤  1.0
Zoek een goede oplossing.


In [26]:
#SOLUTION_START

class OptimalisatieProblem(Annealer):
    def energy(self):
        x = self.state[0]
        y = self.state[1]
        return -(0.2 + x**2 + y**2 - 0.1 * math.cos(6*math.pi*x) - 0.1 * math.cos(6*math.pi*y))   # - want max

    def move(self):
        i = np.random.randint(0,2)
        self.state[i] += np.random.normal(0, 0.1)
        self.state[i] = np.clip(self.state[i], -1, 1)

init_opl =  np.random.uniform(-1,1, size=2) #initial solution
opdracht = OptimalisatieProblem(init_opl)
opdracht.anneal()

#SOLUTION_END

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


(array([ 1., -1.]), -2.0)